
# Project 2 Code
Authors: Lily Geiser, Meredith Lou, Rich Pihlstrom

## Datasets
The datasets for our project are the following:

1. [USA Tech Companies Stats](https://www.kaggle.com/datasets/lamiatabassum/top-50-us-tech-companies-2022-2023-dataset/data): data about the top 50 tech companies in the US for 2022-23. File: companies.csv
2. [Per capita energy-related carbon dioxide emissions by state](https://www.eia.gov/environment/emissions/state/): data from the US Energy Information Administrations (EIA) about the per capita CO2 emissions per US state from 1970-2021. File: emissions.csv
3. [NASDAQ-100 Stock Price Data](https://www.kaggle.com/datasets/kalilurrahman/nasdaq100-stock-price-data): data about stock prices of all NASDAQ-100 index stocks from 2010-21. File: stocks.csv

## Cleaning
The code in this file cleans and process each of the three datasets. Important modifcations include converting columns to appropriate datatypes, melting columns for easier use, and imputing new columns based on the data.

#### Initializing libraries and output function

In [382]:
import pandas as pd
def df_info(df):
    print(f"{'#'*3} COLUMN INFO {'#'*90}")
    print(df.info())
    print(f"{'#'*3} 5 RANDOM ROWS {'#'*88}")
    print(df.sample(5))
    print("#"*56)

#### Read in data

In [383]:
files = ["companies","emissions","stocks"]
dfs = {}
for file in files:
    dfs[file] = pd.read_csv(f"data/raw/{file}.csv")
    dfs[file] = dfs[file].rename(columns = {col:col.lower() for col in dfs[file].columns})

### Stocks

The output below gives an overview of our stock data. First, we notice that columns like "date" are not formattes as datetime object. Second, we can see that the data is stored for small time increments. Other data that we have is by year, so we will aggregate this data to match. Finally, this data is a little bit hard to understand by itself, so we will impute some columns based on this data which we can use a bit more easily.

In [384]:
df_info(dfs["stocks"])

### COLUMN INFO ##########################################################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271680 entries, 0 to 271679
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   date       271680 non-null  object 
 1   open       271680 non-null  float64
 2   high       271680 non-null  float64
 3   low        271680 non-null  float64
 4   close      271680 non-null  float64
 5   adj close  271680 non-null  float64
 6   volume     271680 non-null  int64  
 7   name       271680 non-null  object 
dtypes: float64(5), int64(1), object(2)
memory usage: 16.6+ MB
None
### 5 RANDOM ROWS ########################################################################################
              date       open       high        low      close  adj close  \
82733   2012-07-24  37.529999  37.639999  37.110001  37.520000  33.036900   
135269  2016-07-01  32.639999  32.889999  32

We cast the date column as a datetime object. Using this new formatting, we can create a new column that just contains the year of the date—this will help us aggregate. Finally, we case-fold the name column following a preference to have all string columns be in lowercase.

In [385]:
dfs["stocks"]["date"] = pd.to_datetime(dfs["stocks"]["date"])
dfs["stocks"]["year"] = dfs["stocks"]["date"].dt.year
dfs["stocks"]["name"] = dfs["stocks"]["name"].str.lower()

Using the new "year" column, we aggregate the stock data by both "name" and "year." This allows us to compute the yearly metrics of "high," "low," and "change_in_close." These represent the highest price, lowest price, and change is closing price over the year, respectively. We replace our stock data with this imputed data, as it not only is aggregated in a more informative time granularity given the context of our data, but also is a bit easier to understand. NB: we changed the stock "name" to stock "code." This allows clear distinction across all of our data for the full company name and its stock representation.

In [386]:
rows = []
for tup, df in dfs["stocks"].groupby(["name","year"]):
    rows.append([tup[0],tup[1],max(df["high"]),min(df["low"]),df["close"].iloc[-1]-df["close"].iloc[0]])
dfs["stocks"] = pd.DataFrame(rows, columns = ["code","year","high","low","change_in_close"])

We see now that our columns are appropriate datatypes and that our stock data is redefined.

In [387]:
df_info(dfs["stocks"])

### COLUMN INFO ##########################################################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1120 entries, 0 to 1119
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   code             1120 non-null   object 
 1   year             1120 non-null   int64  
 2   high             1120 non-null   float64
 3   low              1120 non-null   float64
 4   change_in_close  1120 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 43.9+ KB
None
### 5 RANDOM ROWS ########################################################################################
      code  year        high         low  change_in_close
36     adp  2010   41.413521   23.230904         3.028973
196   biib  2014  361.929993  270.619995        59.120026
1099   xel  2016   45.419998   35.189999         5.000000
187   bidu  2017  274.970001  165.820007        65.910004


### Companies
The output below gives an overview of our companny data. The most notable thing about this dataset is the clunky naming of columns. We will replace these with more usable names.

In [388]:
df_info(dfs["companies"])

### COLUMN INFO ##########################################################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   company name                                      50 non-null     object 
 1   industry                                          50 non-null     object 
 2   sector                                            50 non-null     object 
 3   hq state                                          50 non-null     object 
 4   founding year                                     50 non-null     int64  
 5   annual revenue 2022-2023 (usd in billions)        50 non-null     float64
 6   market cap (usd in trillions)                     50 non-null     float64
 7   stock name                                        50 non-null     object 


In [389]:
list(dfs["companies"].columns)

['company name',
 'industry',
 'sector',
 'hq state',
 'founding year',
 'annual revenue 2022-2023 (usd in billions)',
 'market cap (usd in trillions)',
 'stock name',
 'annual income tax in 2022-2023 (usd in billions)',
 'employee size']

Send all object columns to lowercase.

In [390]:
for i, col in enumerate(dfs["companies"].columns):
    if dfs["companies"].dtypes[i] == object:
        dfs["companies"][col] = dfs["companies"][col].str.lower()

As with the stock data, we rename the "company name" and "stock name" columns as "name" and "code," respectively.

In [391]:
print(list(dfs["companies"]["company name"].unique()))
print(list(dfs["companies"]["stock name"].unique()))

dfs["companies"] = dfs["companies"].rename(columns = {"company name":"name",
                                                      "stock name":"code"})

['apple inc.', 'microsoft corporation', 'alphabet (google)', 'amazon', 'nvidia corporation', 'tesla', 'meta platforms', 'broadcom inc.', 'oracle corporation', 'cisco systems inc.', 'salesforce inc.', 'adobe inc.', 'texas instruments inc.', 'advanced micro devices (amd) inc.', 'qualcomm inc.', 'netflix', 'intel corporation', 'intuit inc.', 'ibm corporation', 'applied materials inc.', 'booking holdings', 'analog devices inc.', 'servicenow inc.', 'automatic data processing', 'paypal holdings inc.', 'airbnb', 'fiserv inc.', 'lam research corporation', 'uber technologies inc.', 'micron technology', 'equinix', 'activision blizzard', 'palo alto networks inc.', 'synopsys inc.', 'cadence design systems inc.', 'kla corporation', 'arista networks inc.', 'vmware inc.', 'workday inc.', 'fortinet inc.', 'block inc.', 'snowflake inc.', 'roper technologies', 'microchip technology inc.', 'autodesk inc.', 'globalfoundries', 'iqvia holdings', 'marvell technology inc.', 'dell technologies inc.', 'hp inc.'

Likewise, we rename six other columns for easier use with querying.

In [392]:
dfs["companies"] = dfs["companies"].rename(columns = {
    "hq state":"state",
    "founding year":"founded",
    "annual revenue 2022-2023 (usd in billions)":"revenue_22_23_USD_e9",
    "market cap (usd in trillions)":"market_cap_USD_e12",
    "annual income tax in 2022-2023 (usd in billions)":"incomeTax_22_23_USD_e9",
    "employee size":"emp_num"
})

We can see that our new company data has much more legible column headers.

In [393]:
df_info(dfs["companies"])

### COLUMN INFO ##########################################################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    50 non-null     object 
 1   industry                50 non-null     object 
 2   sector                  50 non-null     object 
 3   state                   50 non-null     object 
 4   founded                 50 non-null     int64  
 5   revenue_22_23_USD_e9    50 non-null     float64
 6   market_cap_USD_e12      50 non-null     float64
 7   code                    50 non-null     object 
 8   incomeTax_22_23_USD_e9  50 non-null     float64
 9   emp_num                 50 non-null     int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 4.0+ KB
None
### 5 RANDOM ROWS #####################################################################

### Emissions
The output below gives an overview of our per capita emissions data. This data is formatted such that each year has its own column. This is super annoying to use, so we will simply melt the columns appropriately.

In [394]:
dfs["emissions"].sample(5)

,state,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2016,2017,2018,2019,2020,2021,change_1970_2021_pct,change_1970_2021_abs,change_2020_2021_pct,change_2020_2021_abs
16,Kansas,22.7,23.5,24.9,25.2,25.3,25.2,26.3,26.7,29.0,...,21.5,20.3,21.6,20.8,19.7,20.3,-10.44%,-2.4,3.42%,0.7
18,Louisiana,39.5,39.5,41.3,44.6,45.1,41.1,46.2,51.0,52.2,...,40.4,41.4,41.8,41.8,39.4,40.8,3.29%,1.3,3.43%,1.3
26,Montana,20.6,20.9,21.8,23.4,22.5,22.5,25.7,26.8,27.2,...,29.9,29.6,29.4,30.2,24.2,25.8,25.29%,5.2,6.67%,1.6
0,Alabama,29.7,28.2,29.7,30.6,30.0,29.3,28.9,29.5,27.8,...,23.4,22.3,23.0,21.6,19.6,21.5,-27.84%,-8.3,9.70%,1.9
36,Oklahoma,21.6,21.5,22.3,22.2,22.6,22.5,24.4,25.2,26.4,...,24.5,23.8,25.0,23.1,21.3,22.0,1.63%,0.4,3.40%,0.7


Send all state names to lowercase.

In [395]:
dfs["emissions"]["state"] = dfs["emissions"]["state"].str.lower()

The final row of the data gives the average across all states.

In [396]:
dfs["emissions"].tail(3)

,state,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2016,2017,2018,2019,2020,2021,change_1970_2021_pct,change_1970_2021_abs,change_2020_2021_pct,change_2020_2021_abs
49,wisconsin,19.9,19.0,18.7,18.9,18.1,17.6,18.3,18.8,19.1,...,16.6,17.0,17.4,16.3,14.8,15.7,-21.02%,-4.2,6.51%,1.0
50,wyoming,55.7,55.3,62.4,66.9,65.3,66.3,77.7,87.9,85.7,...,104.7,108.3,110.3,101.9,96.2,94.3,69.38%,38.6,-2.02%,-1.9
51,average all states,20.7,20.7,21.5,22.2,21.2,20.5,21.5,21.9,21.9,...,16.0,15.8,16.2,15.7,13.9,14.8,-28.67%,-5.9,6.72%,0.9


This isn't very useful, as we will probably be merging on the state. As such, we can just drop the column.

In [397]:
dfs["emissions"] = dfs["emissions"].iloc[:-1]
dfs["emissions"].tail(3)

,state,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2016,2017,2018,2019,2020,2021,change_1970_2021_pct,change_1970_2021_abs,change_2020_2021_pct,change_2020_2021_abs
48,west virginia,44.0,44.0,49.5,54.0,55.3,53.1,55.4,54.0,50.3,...,51.6,49.9,49.4,47.6,43.0,49.5,12.61%,5.5,15.25%,6.5
49,wisconsin,19.9,19.0,18.7,18.9,18.1,17.6,18.3,18.8,19.1,...,16.6,17.0,17.4,16.3,14.8,15.7,-21.02%,-4.2,6.51%,1.0
50,wyoming,55.7,55.3,62.4,66.9,65.3,66.3,77.7,87.9,85.7,...,104.7,108.3,110.3,101.9,96.2,94.3,69.38%,38.6,-2.02%,-1.9


We melt the columns here. Additionally, we are not super interested in the columns about percent-change over certain years, so we can also drop those columns. When we melt the data, we leave the "state" column alone, send the column headers (each year) to the "year" column, and the corresponding values per State-Year to the "emissions_per_cap" column.

In [398]:
dfs["emissions"] = dfs["emissions"][dfs["emissions"].columns[:-4]].melt(id_vars = "state",
                                                                      var_name = "year",
                                                                      value_name = "emissions_per_cap")

We can see that we have substantially reduced the number of columns.

In [399]:
df_info(dfs["emissions"])

### COLUMN INFO ##########################################################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2652 entries, 0 to 2651
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   state              2652 non-null   object 
 1   year               2652 non-null   object 
 2   emissions_per_cap  2652 non-null   float64
dtypes: float64(1), object(2)
memory usage: 62.3+ KB
None
### 5 RANDOM ROWS ########################################################################################
                     state  year  emissions_per_cap
518   district of columbia  1980                8.2
1419             tennessee  1997               22.3
370               illinois  1977               22.5
1472                  utah  1998               29.3
1810              missouri  2005               25.0
########################################################


We then simply output each cleaned file.

In [400]:
for file in files:
    dfs[file].to_csv(f"data/cleaned/{file}.csv", index = False)